<a href="https://colab.research.google.com/github/Himabindugssn/GenuineMetrics/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=3e43be27a53aefba8e930fa2abdcba95196890f0f154aa97826850d1434e1b88
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
# importing the library for the analysis 
import pandas as pd 
import numpy as np
import glob 
import os 
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import TfidfVectorizer
#from keras import Sequential
import emoji
import re

#from google.colab import drive
#drive.mount('/content/gdrive')
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
twitter = pd.read_csv('/content/drive/My Drive/master_twitter_01.csv')


In [ ]:
twitter.shape

In [ ]:
twitter.columns

In [ ]:
twitter.head()

In [ ]:
twitter= twitter.drop_duplicates(subset="Contents")

In [ ]:
twitter_emoji = pd.read_excel('/content/drive/My Drive/GSTN/Emoji_Popularity_index.xlsx')

In [ ]:
# Getting the emoji for the tweets 
emoj_list =emoji.UNICODE_EMOJI
print(len(emoj_list))



In [ ]:
# checking the emoji mapping and their popularity index .....


emoji_pop  = {}
for row in range(len(twitter_emoji)) : 
    #print(twitter_emoji.loc[row, "Emoji"],twitter_emoji.loc[row, "Polarity"])
    emoji_pop[twitter_emoji.loc[row, "Emoji"]] = twitter_emoji.loc[row, "Polarity"]

print(emoji_pop)

counter = 0
for emoj in emoj_list: # emoj list is prepared from the emoji library 
    if emoj in emoji_pop.keys():
        val = emoji_pop[emoj]
        counter= counter + 1
        
print("Total emoji which got matched are ", counter )


In [ ]:
# User Defined function : Data cleansing and features creations .......

def ind_creation(x):
    if x =="POSITIVE":
        return 1
    elif x  == "NEGATIVE":
        return 0 
    else:
        return -1

# Creating the indicator for the further analysis .....
twitter['Ind_Sentiment'] = twitter['Sentiment'].map(ind_creation) # calling the function which is created in the above 

# We would be dealing with indicator of 1 and 0 only ......
twitter = twitter[twitter['Ind_Sentiment'] > -1 ]


#checking the distribution of the indicator created for the analysis .....
print(twitter['Ind_Sentiment'].value_counts())

In [ ]:
# Creating the count of emoji for the analysis .....
def split_count(text):
    emoji_list = []
    text = str(text)
    pat = "[^\w\s,]"
    if len(text)>0:
        data = re.findall(pat, text)
        for word in data:
            for char1 in word: 
                if char1 in emoj_list:     
                    emoji_list.append(word)
                else:
                    continue 
    else:
        emoji_list.append(0)
    return(len(emoji_list))

# Creating the field for the further analysis 
twitter['cnt_emoji'] = twitter['Contents'].map(split_count)

print(twitter['cnt_emoji'].value_counts())

In [ ]:
# checking the age for the further creations  
print(twitter['Gender'].unique())

# Using the gender for the classification 

def gender(x):
    x = str(x)
    if x.upper() == 'MALE':
        return('MALE')
    elif x.upper() == 'FEMALE':
        return('FEMALE')
    else:
        return('UNK')


# Creating the indicator variable for the analysis 
def ind_create(x,y):
    if x.upper() == y.upper():
        return 1
    else:
        return 0

# Creating the three indicators variables for the analysis .....

twitter['Gender'] = twitter['Gender'].map(gender)
twitter['ind_male']= twitter['Gender'].map(lambda x :ind_create(x,'MALE'))
twitter['ind_female']= twitter['Gender'].map(lambda x :ind_create(x,'FEMALE'))
twitter['ind_unk']= twitter['Gender'].map(lambda x :ind_create(x,'UNK'))

print(twitter['Gender'].value_counts())


In [ ]:
# Creating the total word counts of the tweet for the analysis .....
def word_count(text):
    text = str(text)
    word_list = []
    data = text.split(" ")
    return(len(data))

twitter['word_count']= twitter['Contents'].map(word_count)

# checking the counts of the word counts 
print(twitter["word_count"].value_counts())

In [ ]:
def val(emoj):
    if emoj in emoji_pop.keys():
        val = int(emoji_pop[emoj])
    else:
        val = 0
    return(val)

# Creating the total popularity index for the emoji for the analysis ....

def popularity_index(text):
    pop_scr = []
    try:
        text = str(text).encode('latin-1').decode('utf-8').strip()
        
        pat = "[^\w\s,]"
        if len(text)> 0:
            data = re.findall(pat, text)
            for word in data:
                for char1 in word: 
                    if char1 in emoj_list :
                        pop_scr.append(val(char1)) # storing the total popularity of the emoji used
                    else:
                        continue
    except UnicodeDecodeError:
        pop_scr.append(0)
    # finding the sum of the list for the analysis ...
    return(sum(pop_scr))

# checking the emoji popularity variables for the further analysis ...
twitter['emoji_popularity'] = twitter['Contents'].map(popularity_index)

print(twitter['emoji_popularity'].value_counts())

In [ ]:
################ Creating the function for generating some additional features  ##################

    
# Creating the function to identify max popularity given to tweets 

def popularity_strength(text):    
    pop_scr = []
    try:
        text = str(text).encode('latin-1').decode('utf-8').strip() 
        pat = "[^\w\s,]"
        if len(text)> 0:
            data = re.findall(pat, text)
            for word in data:
                for char1 in word: 
                    if char1 in emoj_list :
                        pop_scr.append(val(char1)) # storing the total popularity of the emoji used
                    else:
                        continue
    except UnicodeDecodeError:
        pop_scr.append(0) 
    # finding the sum of the list for the analysis ...
    if len(pop_scr) ==0:
        return(0)
    elif len(pop_scr)==1:
        return((pop_scr[0]))
    else:
        max_pop = max(pop_scr)
        min_pop = min(pop_scr)
        return((max_pop + min_pop))

# Creating the field for the further analysis Manx values for the popularity index ...
twitter['emoji_strength'] = twitter['Contents'].map(popularity_strength)
print(twitter['emoji_strength'].value_counts())

In [ ]:
# Further cleaning of the data for creating the embedding ......
twitter['Contents'] = twitter['Contents'].apply(lambda x: str(x))                           # convert everything to lower case
twitter['Contents'] = twitter['Contents'].apply(lambda x: x.lower())                           # convert everything to lower case
twitter['Contents'] = twitter['Contents'].apply(lambda x: re.sub(r'[@]\w+', '', x))            # remove usertags 
twitter['Contents'] = twitter['Contents'].apply(lambda x: re.sub(r'[#\(\)\[\]]', '', x))       # remove brackets and # from hashtags
twitter['Contents'] = twitter['Contents'].apply(lambda x: re.sub(r'https?[\w./:]+', '', x))    # remove urls
twitter['Contents'] = twitter['Contents'].apply(lambda x: re.sub(r'\.{2,}', ' ', x))           # replacing 2+ dots to space
twitter['Contents'] = twitter['Contents'].apply(lambda x: re.sub(r'(.)\1+', r'\1\1', x))       # replace multiple chars to 2 chars
twitter['Contents'] = twitter['Contents'].apply(lambda x: re.sub(r'(-|\')', '', x))            # removing -|\''
twitter['Contents'] = twitter['Contents'].apply(lambda x: x.strip())                           # remove trailing spaces


In [ ]:

# Subsetting the data which we need to use only for the analysis ...

X=twitter.loc[:,['Contents','cnt_emoji','word_count','ind_male','ind_female','ind_unk','Followers','Following','emoji_popularity','emoji_strength']]
y=twitter['Ind_Sentiment']


In [ ]:
# Splitting the data for the train and test 
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Checking the data for the further analysis 
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)
print(ytest.count(),ytest.sum(), ytest.count() - ytest.sum())

In [ ]:
tokenizer = TweetTokenizer()
def tokenize(tweet):
    try:
        tweet = tweet.lower()
        tokens = tokenizer.tokenize(tweet)
        tokens = list(filter(lambda t: not t.startswith('@'), tokens))
        tokens = list(filter(lambda t: not t.startswith('#'), tokens))
        tokens = list(filter(lambda t: not t.startswith('http'), tokens))
        return tokens
    except:
        return 'NC'

In [ ]:
vocab_size = 1000

vectorizer = CountVectorizer(stop_words='english', max_features=vocab_size, tokenizer=tokenize)
vectorizer.fit(Xtrain.loc[:,'Contents'])

In [ ]:
train_embeddings=vectorizer.transform(Xtrain.loc[:,'Contents'])
test_embeddings = vectorizer.transform(Xtest.loc[:,'Contents'])

In [ ]:
# We have to add the standarized values for additional features which we have created ...
mean_train = Xtrain.loc[:,'cnt_emoji'].mean()
std_train = Xtrain.loc[:,'cnt_emoji'].std()

mean_train_cnt = Xtrain.loc[:,'word_count'].mean()
std_train_cnt = Xtrain.loc[:,'word_count'].std()

# Mean Count of the "followers" & "following"
mean_train_f = Xtrain.loc[:,'Followers'].mean()
std_train_f = Xtrain.loc[:,'Followers'].std()

mean_train_f1 = Xtrain.loc[:,'Following'].mean()
std_train_f1 = Xtrain.loc[:,'Following'].std()


mean_train_p1 = Xtrain.loc[:,'emoji_popularity'].mean()
std_train_p1 = Xtrain.loc[:,'emoji_popularity'].std()

mean_train_s1 = Xtrain.loc[:,'emoji_strength'].mean()
std_train_s1 = Xtrain.loc[:,'emoji_strength'].std()


# Standarizing the emoji count in eaach tweets for the further ....
Xtrain['std_cnt_emoji'] = Xtrain['cnt_emoji'].map(lambda x : (x - mean_train)/std_train)
Xtest['std_cnt_emoji'] = Xtest['cnt_emoji'].map(lambda x : (x - mean_train)/std_train)

Xtrain['std_cnt_word'] = Xtrain['word_count'].map(lambda x : (x - mean_train_cnt)/std_train_cnt)
Xtest['std_cnt_word'] = Xtest['word_count'].map(lambda x : (x - mean_train_cnt)/std_train_cnt)


Xtrain['std_followers'] = Xtrain['Followers'].map(lambda x: (x- mean_train_f)/std_train_f)
Xtest['std_followers'] = Xtest['Followers'].map(lambda x: (x- mean_train_f)/std_train_f)

Xtrain['std_following'] = Xtrain['Following'].map(lambda x: (x- mean_train_f1)/std_train_f1)
Xtest['std_following'] = Xtest['Following'].map(lambda x: (x- mean_train_f1)/std_train_f1)

Xtrain['std_following'] = Xtrain['Following'].map(lambda x: (x- mean_train_f1)/std_train_f1)
Xtest['std_following'] = Xtest['Following'].map(lambda x: (x- mean_train_f1)/std_train_f1)

Xtrain['std_emoji_pop'] = Xtrain['emoji_popularity'].map(lambda x: (x- mean_train_p1)/std_train_p1)
Xtest['std_emoji_pop'] = Xtest['emoji_popularity'].map(lambda x: (x- mean_train_f1)/std_train_f1)

Xtrain['std_emoji_strength'] = Xtrain['emoji_strength'].map(lambda x: (x- mean_train_s1)/std_train_s1)
Xtest['std_emoji_strength'] = Xtest['emoji_strength'].map(lambda x: (x- mean_train_s1)/std_train_s1)

print("Total ")
print(Xtrain.columns)


print(Xtrain['std_cnt_emoji'].shape)

In [ ]:
from scipy.sparse import hstack

print(" checking train data .....")

Xtrain_embedded = hstack((train_embeddings,np.array(Xtrain['std_cnt_emoji'])[:,None])).tocsr()
Xtrain_embedded = hstack((Xtrain_embedded,np.array(Xtrain['std_cnt_word'])[:,None])).tocsr()
Xtrain_embedded = hstack((Xtrain_embedded,np.array(Xtrain['std_followers'])[:,None])).tocsr()
Xtrain_embedded = hstack((Xtrain_embedded,np.array(Xtrain['std_following'])[:,None])).tocsr()
Xtrain_embedded = hstack((Xtrain_embedded,np.array(Xtrain['ind_male'])[:,None])).tocsr()
Xtrain_embedded = hstack((Xtrain_embedded,np.array(Xtrain['ind_female'])[:,None])).tocsr()
Xtrain_embedded = hstack((Xtrain_embedded,np.array(Xtrain['ind_unk'])[:,None])).tocsr()
Xtrain_embedded = hstack((Xtrain_embedded,np.array(Xtrain['std_emoji_pop'])[:,None])).tocsr()
Xtrain_embedded = hstack((Xtrain_embedded,np.array(Xtrain['std_emoji_strength'])[:,None])).tocsr()


print("checking test data ....")
Xtest_embedded = hstack((test_embeddings,np.array(Xtest['std_cnt_emoji'])[:,None])).tocsr()
Xtest_embedded = hstack((Xtest_embedded,np.array(Xtest['std_cnt_word'])[:,None])).tocsr()
Xtest_embedded = hstack((Xtest_embedded,np.array(Xtest['std_followers'])[:,None])).tocsr()
Xtest_embedded = hstack((Xtest_embedded,np.array(Xtest['std_following'])[:,None])).tocsr()
Xtest_embedded = hstack((Xtest_embedded,np.array(Xtest['ind_male'])[:,None])).tocsr()
Xtest_embedded = hstack((Xtest_embedded,np.array(Xtest['ind_female'])[:,None])).tocsr()
Xtest_embedded = hstack((Xtest_embedded,np.array(Xtest['ind_unk'])[:,None])).tocsr()
Xtest_embedded = hstack((Xtest_embedded,np.array(Xtest['std_emoji_pop'])[:,None])).tocsr()
Xtest_embedded = hstack((Xtest_embedded,np.array(Xtest['std_emoji_strength'])[:,None])).tocsr()

print(Xtest_embedded.shape) 
print(type(Xtest_embedded))

print(Xtrain_embedded.shape) 
print(type(Xtrain_embedded))


In [ ]:
twitter.shape

In [ ]:
print(Xtrain_embedded.shape, ytrain.shape)
ytrain_01 = ytrain.values.reshape(-1,1)
print(ytrain_01.shape)

In [ ]:
# Building the model for the further analysis we have 
from keras.models import Sequential
from keras import layers
from keras.layers import *

In [ ]:
print(np.isnan(Xtrain_embedded.data).any())
Xtrain_embedded.data[np.isnan(Xtrain_embedded.data)] = 0.001 # Missing value treatment 
print(Xtrain_embedded.shape)
print(np.isnan(Xtrain_embedded.data).any())

In [ ]:
#print(Xtrain_embedded.shape) 


# Building the model for the further analysis we have 
from keras.models import Sequential
from keras import layers
from keras.layers import *

# This is ANN model with 4  layers  with additional features 9 
model_01=Sequential()
model_01.add(Dense(128, input_dim=1009))
model_01.add(BatchNormalization())
model_01.add(Activation('relu'))
model_01.add(layers.Dropout(0.4))

model_01.add(Dense(64, activation='relu'))
model_01.add(BatchNormalization())
model_01.add(layers.Dropout(0.4))
model_01.add(Dense(32, activation='relu'))
model_01.add(BatchNormalization())
model_01.add(layers.Dropout(0.4))
model_01.add(Dense(16, activation='relu'))
model_01.add(BatchNormalization())
model_01.add(layers.Dropout(0.4))
model_01.add(Dense(8, activation='relu'))
model_01.add(BatchNormalization())
model_01.add(layers.Dropout(0.4))
model_01.add(Dense(1, activation='sigmoid'))
model_01.summary()
#model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model_01.compile(optimizer='rmsProp',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
# This is model without considering the additional features which has been created ....

hist_01=model_01.fit(Xtrain_embedded.todense(),ytrain_01, epochs=50, batch_size= 512,validation_split= 0.2)
history_01_dict = hist_01.history

In [ ]:
print(Xtest.shape)
print(ytest.shape)

# Evaluating the model for the ana;lysis we have 
test_loss, test_acc = model_01.evaluate(Xtest_embedded.todense(), ytest)

# Printing the test loss and test accuracy 
print(test_loss,test_acc)


In [ ]:
#Run these codes first in order to install the necessary libraries and perform authorization
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
#Mount your Google Drive:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
#After success run Drive FUSE program, you can create a directory Sentiment_Analysis and access your drive at /content/drive with using command
import os
os.mkdir("/content/drive/Sentiment_Analysis")
os.chdir("/content/drive/")
!ls

In [ ]:
#Append your path
import sys
sys.path.append('/content/drive/Sentiment_Analysis')

In [ ]:
#Now save the model in required directory
model.save('/content/drive/Sentiment_Analysis/sentiment_analysis_model_new.h5')
print("Saved model to disk")

In [ ]:
#Check the content of the directory
os.chdir("/content/drive/Sentiment_Analysis")
!ls

In [ ]:
#Code to load the saved model
model = load_model('/content/drive/Sentiment_Analysis/sentiment_analysis_model_new.h5')
print("Model Loaded")